In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

# Clustering
from sklearn.cluster import KMeans, DBSCAN
from sklearn import preprocessing
from sklearn.metrics import silhouette_score

# Dimensionality reduction
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

# Visualization
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.axes3d as p3
from matplotlib import animation

In [4]:
df = pd.read_csv("Pharmacy Claims, Eligibility.csv")

C:\Users\paliwam\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (87,88,89,127,146,155) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6929246 entries, 0 to 6929245
Columns: 170 entries, Active Ingredient Count to Zip Code (Elig)
dtypes: float64(47), int64(32), object(91)
memory usage: 42.8 GB


In [6]:
def sample_first_prows(data, perc=0.7):
    import pandas as pd
    return data.head(int(len(data)*(perc)))

df_play = sample_first_prows(df, perc = 0.1)

In [7]:
print(df_play.dtypes)

Active Ingredient Count       int64
Adjudication Status          object
Age (risk)                  float64
AWP Amount                  float64
Benefit Class (Elig)         object
                             ...   
Usual And Customary Cost    float64
Vaccine Admin Fee           float64
Work County Code            float64
Work County Code (Elig)     float64
Zip Code (Elig)             float64
Length: 170, dtype: object


In [8]:
#Checking data types of each column
pd.set_option('display.max_rows', 200)
df_play.dtypes

Active Ingredient Count                   int64
Adjudication Status                      object
Age (risk)                              float64
AWP Amount                              float64
Benefit Class (Elig)                     object
Benefit Code                             object
BI Coverage Types                        object
BI Coverage Types (Elig)                 object
BI Member ID                             object
BI Member ID (Elig)                      object
Bill Type                               float64
BIMemberID (risk)                        object
Birth Date (Elig)                        object
Brand/Generic Flag                       object
Cancer Treatment                        float64
Carrier Provider ID                       int64
Check Amount                            float64
Check Date                               object
Check Number                            float64
Chronic Condition Count                 float64
City (Elig)                             

In [9]:
# number of muissing values in each coumn 
df.isna().sum()* 100 / len(df)

Active Ingredient Count                   0.000000
Adjudication Status                       0.000000
Age (risk)                                0.059141
AWP Amount                                0.000000
Benefit Class (Elig)                      0.045705
Benefit Code                              0.000000
BI Coverage Types                         0.000000
BI Coverage Types (Elig)                  0.045705
BI Member ID                              0.000000
BI Member ID (Elig)                       0.045705
Bill Type                               100.000000
BIMemberID (risk)                         0.059141
Birth Date (Elig)                         0.045705
Brand/Generic Flag                        0.000592
Cancer Treatment                          0.059141
Carrier Provider ID                       0.000000
Check Amount                              0.000000
Check Date                                0.000000
Check Number                            100.000000
Chronic Condition Count        

In [16]:
df_play.describe()

,Active Ingredient Count,Age (risk),AWP Amount,Bill Type,Cancer Treatment,Carrier Provider ID,Check Amount,Check Number,Chronic Condition Count,Claim Dependent Identifier,...,Service Counter,Service Line Index,Service Year,Subsection Code (Elig),Total Charge Amount,Usual And Customary Cost,Vaccine Admin Fee,Work County Code,Work County Code (Elig),Zip Code (Elig)
count,692924.000000,692343.000000,692924.000000,0.0,692343.000000,6.929240e+05,692924.000000,0.0,692343.000000,692924.000000,...,692924.000000,692924.0,692924.0,530452.000000,692924.000000,692924.000000,692924.0,528201.000000,528193.000000,6.925440e+05
mean,10.587434,52.298433,222.017476,NaN,0.022347,1.496640e+09,165.899801,NaN,4.707382,0.506650,...,0.949303,1.0,2019.0,159.756845,176.829016,380.459662,0.0,31.399844,31.400990,9.989665e+07
std,7.386294,20.018775,2043.946499,NaN,0.147810,2.820724e+08,1053.629695,NaN,3.855516,0.698723,...,0.288476,0.0,0.0,249.371201,1061.032658,2199.152583,0.0,18.872753,18.872601,1.511999e+08
min,0.000000,0.000000,0.000000,NaN,0.000000,1.003005e+09,-38006.380000,NaN,0.000000,0.000000,...,-1.000000,1.0,2019.0,0.000000,-38106.380000,-81678.200000,0.0,0.000000,0.000000,0.000000e+00
25%,5.000000,40.000000,9.840000,NaN,0.000000,1.275637e+09,0.000000,NaN,2.000000,0.000000,...,1.000000,1.0,2019.0,0.000000,2.660000,29.950000,0.0,13.000000,13.000000,3.244300e+04
50%,9.000000,55.000000,28.590000,NaN,0.000000,1.497795e+09,0.000000,NaN,4.000000,0.000000,...,1.000000,1.0,2019.0,1.000000,7.240000,78.590000,0.0,37.000000,37.000000,3.346900e+04
75%,14.000000,66.000000,72.990000,NaN,0.000000,1.730227e+09,22.760000,NaN,7.000000,1.000000,...,1.000000,1.0,2019.0,226.000000,36.310000,238.990000,0.0,48.000000,48.000000,3.230389e+08
max,62.000000,107.000000,93340.800000,NaN,1.000000,1.992966e+09,114219.140000,NaN,29.000000,2.000000,...,1.000000,1.0,2019.0,999.000000,114319.140000,245024.600000,0.0,99.000000,99.000000,9.837100e+08


In [17]:
df_play.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692924 entries, 0 to 692923
Columns: 170 entries, Active Ingredient Count to Zip Code (Elig)
dtypes: float64(47), int64(32), object(91)
memory usage: 898.7+ MB


In [54]:
dups = df_play.pivot_table(index = ['Transaction Date'], aggfunc ='size')
print (df['Transaction Date'].min())
print (df['Transaction Date'].max())

1/1/2019
9/9/2019


In [49]:
grouped_multiple = df.groupby(['Provider Name', 'Drug Category']).agg({'Drug Category': ['size']})